In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, timedelta
from pathlib import Path

import copy
import pyarrow.parquet as pq

In [2]:
df_train = pd.read_csv('../data/train.csv')

In [3]:
data_path = Path('../data')
data_path.exists()

True

<h3> group stocks by their correlation</h3>

In [4]:
stock_list = df_train['stock_id'].unique().tolist()
print(len(stock_list), stock_list[:2])

112 [0, 1]


In [5]:
df_corr_target = pd.read_excel('../data/y_1/target_corr.xlsx', engine='openpyxl')

In [6]:
df_corr_target[df_corr_target['corr_norm_log']>=0.75]

,stock_1,stock_2,corr,corr_norm,corr_norm_log
1,0,2,0.836528,0.836528,0.781227
12,0,14,0.842872,0.842872,0.755641
14,0,16,0.884037,0.884037,0.762516
15,0,17,0.903870,0.903870,0.771743
18,0,20,0.873712,0.873712,0.762998
...,...,...,...,...,...
6197,119,123,0.907542,0.907542,0.770939
6199,119,125,0.868282,0.868282,0.783312
6209,122,126,0.859494,0.859494,0.766255
6211,123,125,0.915465,0.915465,0.768480


In [7]:
sel_by = (df_corr_target['corr_norm_log']>=0.8) & (df_corr_target['corr']>=0.85) & (df_corr_target['corr_norm']>=0.85)
high_corr_list = df_corr_target[sel_by][['stock_1', 'stock_2']].values.reshape(-1)
high_corr_list = list(set(high_corr_list))
high_corr_list.sort()

In [8]:
print(f'{len(high_corr_list)}\n{high_corr_list}')

28
[10, 11, 13, 14, 15, 23, 29, 32, 35, 42, 43, 50, 52, 62, 69, 70, 72, 73, 76, 87, 93, 95, 101, 108, 109, 119, 122, 126]


In [9]:
sel_by = (df_corr_target['corr_norm_log']<0.5) & (df_corr_target['corr']<0.5) & (df_corr_target['corr_norm']<0.5)
low_corr_list = df_corr_target[sel_by][['stock_1', 'stock_2']].values.reshape(-1)
low_corr_list = list(set(low_corr_list))
low_corr_list.sort()

In [10]:
print(f'{len(low_corr_list)}\n{low_corr_list}')

23
[2, 13, 19, 20, 21, 29, 30, 34, 39, 41, 43, 46, 47, 53, 58, 64, 67, 68, 69, 80, 81, 99, 110]


In [11]:
set(high_corr_list)&set(low_corr_list)

{13, 29, 43, 69}

## load feature importance

In [12]:
df_features=pd.read_excel('../data/y_1/feature_importance_all.xlsx', engine='openpyxl', index_col=0)

In [13]:
df_features.head()

,stock=0,stock_log=0,stock=1,stock_log=1,stock=2,stock_log=2,stock=3,stock_log=3,stock=4,stock_log=4,...,stock_log=122,stock=123,stock_log=123,stock=124,stock_log=124,stock=125,stock_log=125,stock=126,stock_log=126,folder
feature,,,,,,,,,,,,,,,,,,,,,
wap_ema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1
wap_rsi,0.021324,0.028282,0.021427,0.023472,0.007162,0.001897,0.053667,0.052347,0.027419,0.033781,...,0.064445,0.018074,0.020173,0.062432,0.062751,0.003095,0.011206,0.026254,0.033616,z_1
wap_ssma,0.017943,0.014688,0.032834,0.024063,0.018432,0.009381,0.088074,0.059892,0.072510,0.050507,...,0.052301,0.076199,0.050881,0.064353,0.062202,0.029375,0.006786,0.035666,0.035490,z_1
wap_dema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1
wap_tema,0.022806,0.016930,0.035028,0.026020,0.019946,0.010671,0.091798,0.062317,0.068850,0.047974,...,0.053352,0.076512,0.050566,0.068155,0.064572,0.030563,0.007093,0.035439,0.035990,z_1


## select top corr featuers for high corr list

In [14]:
sel_cols = [f'stock={id_}' for id_ in high_corr_list]
df_features[sel_cols].sort_values(by=sel_cols, ascending = [False]*len(high_corr_list))

,stock=10,stock=11,stock=13,stock=14,stock=15,stock=23,stock=29,stock=32,stock=35,stock=42,...,stock=76,stock=87,stock=93,stock=95,stock=101,stock=108,stock=109,stock=119,stock=122,stock=126
feature,,,,,,,,,,,,,,,,,,,,,
wap__absolute_sum_of_changes,0.892771,0.857341,0.889977,0.879028,0.889802,0.894443,0.914817,0.863290,0.884004,0.858732,...,0.904378,0.883758,0.883015,0.883233,0.867318,0.879017,0.893068,0.894052,0.871349,0.842402
wap__mean_abs_change,0.891519,0.823209,0.873563,0.873799,0.878167,0.867619,0.914814,0.862328,0.888634,0.801115,...,0.887486,0.862621,0.864567,0.860076,0.848914,0.876269,0.877818,0.894926,0.803450,0.804353
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",0.891519,0.823209,0.873563,0.873799,0.878167,0.867619,0.914814,0.862328,0.888634,0.801115,...,0.887486,0.862621,0.864567,0.860076,0.848914,0.876269,0.877818,0.894926,0.803450,0.804353
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",0.891245,0.820721,0.870850,0.878007,0.876628,0.867554,0.914193,0.860766,0.887274,0.792822,...,0.885039,0.862107,0.866617,0.857222,0.845446,0.874661,0.877808,0.893555,0.805196,0.802035
real_vol,0.889527,0.852685,0.875589,0.864452,0.883336,0.888077,0.915631,0.880749,0.886140,0.820184,...,0.884542,0.875250,0.867186,0.861267,0.871065,0.880451,0.888045,0.906272,0.856783,0.834651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
price__number_crossing_m__m_-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__ratio_beyond_r_sigma__r_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__count_above__t_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_features.shape

(1626, 225)

In [16]:
top_n = 300
corr_cutoff=0.7
feat_list = []
for stock_id in high_corr_list:
    df_top_n = df_features[f'stock={stock_id}'].sort_values(ascending=False).iloc[:top_n]
    print(np.round(df_top_n[0],4), np.round(df_top_n[-1],4))
    feat_list.append(df_top_n.index.tolist())

0.8928 0.5639
0.8573 0.4289
0.89 0.5449
0.8874 0.6131
0.8898 0.4909
0.8944 0.568
0.9158 0.6709
0.8807 0.6205
0.8886 0.5858
0.8587 0.4306
0.9337 0.7154
0.8559 0.584
0.8916 0.5255
1.0 0.5479
0.8849 0.6541
0.8865 0.4736
0.9856 0.5025
0.8959 0.5599
0.9044 0.5469
0.8838 0.5618
0.883 0.5596
0.8832 0.5437
0.8711 0.4989
0.8805 0.6134
0.8931 0.5257
0.9066 0.5935
0.8713 0.5355
1.0 0.5185


In [17]:
over_lap_feat = feat_list[0]
for i in range(1, len(feat_list)):
    over_lap_feat = set(over_lap_feat) & set(feat_list[i])


In [18]:
print(len(over_lap_feat))
print(over_lap_feat)

70
{'price__fft_aggregated__aggtype_"centroid"', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2', 'wap__fft_coefficient__attr_"abs"__coeff_21', 'wap__fft_coefficient__attr_"abs"__coeff_56', 'wap__fft_coefficient__attr_"abs"__coeff_25', 'price__variation_coefficient', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4', 'wap__fft_coefficient__attr_"abs"__coeff_38', 'wap__fft_coefficient__attr_"abs"__coeff_1', 'wap__fft_coefficient__attr_"abs"__coeff_42', 'wap__fft_coefficient__attr_"abs"__coeff_31', 'wap__fft_coefficient__attr_"abs"__coeff_92', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'price__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4', 'wap__fft_coefficient__attr_"abs"__coeff_19', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0', 'wap__fft_coefficient__attr_"

In [19]:
z1_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_1')].index.tolist()
z2_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_2')].index.tolist()

In [20]:
z1_feats = ['stock_id', 'time_id'] + z1_feats
z2_feats = ['stock_id', 'time_id'] + z2_feats
print(f'{z1_feats}\n{z2_feats}')

['stock_id', 'time_id', 'real_vol', 'wap_std', 'wap__mean_abs_change', 'wap__standard_deviation', 'wap__variation_coefficient', 'wap__absolute_sum_of_changes', 'wap__cid_ce__normalize_False', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.4', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.4', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.4', 'wap

In [21]:
z1_list = []
z2_list = []
for stock_id in [10, 50, 70, 126]:
    df = pd.read_csv(f'../data/z_1/{stock_id}', sep='|', compression='bz2', index_col=0)[z1_feats].copy(deep=True)
    z1_list.append(df)
    df = pd.read_csv(f'../data/z_2/{stock_id}', sep='|', compression='bz2', index_col=0)[z2_feats].copy(deep=True)
    z2_list.append(df)

In [ ]:
df_z1=pd.concat(z1_list, axis=0)
df_z2=pd.concat(z2_list, axis=0)

In [ ]:
df_z1z2 = pd.concat([df_z1, df_z2], axis=1)

In [ ]:
final_feats = ['wap__absolute_sum_of_changes', 
               'wap__fft_aggregated__aggtype_"variance"', 
               #'wap__fft_aggregated__aggtype_"skew"', 
               #'wap__fft_coefficient__attr_"abs"__coeff_69',
               'wap__spkt_welch_density__coeff_5',
               'wap__spkt_welch_density__coeff_8',
               'wap__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
               'price__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 
               'price__variation_coefficient', 
               'price__fft_coefficient__attr_"abs"__coeff_47',
               'price__fft_coefficient__attr_"abs"__coeff_36',
               'price__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',
              ]
final_feats = ['wap__absolute_sum_of_changes', 
               'wap__fft_aggregated__aggtype_"variance"', 
               'wap__fft_coefficient__attr_"abs"__coeff_96',
               'wap__fft_coefficient__attr_"abs"__coeff_90',
               'price__variation_coefficient', 
               'price__mean_abs_change',
               'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
              ]

In [ ]:
corr_z1z2=df_z1z2.corr()
remaining_feats =  corr_z1z2.index.tolist()
for feat in final_feats:
    if feat in remaining_feats:
        remaining_feats = set(remaining_feats) & set(corr_z1z2[corr_z1z2[feat]<0.75].index)


In [ ]:
df_features[df_features.index.isin(remaining_feats)].sort_values(by=['stock=0'],ascending=False)

***
***

## select top corr featuers for low corr list

In [49]:
sel_cols = [f'stock={id_}' for id_ in low_corr_list]
df_features[sel_cols].sort_values(by=sel_cols, ascending = [False]*len(sel_cols))

,stock=2,stock=13,stock=19,stock=20,stock=21,stock=29,stock=30,stock=34,stock=39,stock=41,...,stock=53,stock=58,stock=64,stock=67,stock=68,stock=69,stock=80,stock=81,stock=99,stock=110
feature,,,,,,,,,,,,,,,,,,,,,
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.2",0.902579,0.879613,0.804592,0.896803,0.834363,0.912057,0.817163,0.884417,0.885788,0.884069,...,0.883320,0.821514,0.874531,0.866748,0.865655,0.882768,0.784918,0.791139,0.863228,0.752643
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.0",0.898812,0.878322,0.796007,0.883950,0.830703,0.911815,0.829734,0.883599,0.896010,0.880894,...,0.884049,0.826161,0.874625,0.867279,0.867974,0.882628,0.786051,0.794928,0.867861,0.774888
wap__absolute_sum_of_changes,0.897932,0.889977,0.870573,0.913293,0.862260,0.914817,0.863294,0.894486,0.921835,0.886781,...,0.910097,0.863964,0.886297,0.890381,0.888145,0.883957,0.805853,0.829672,0.880094,0.764091
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_0.8__ql_0.4",0.896543,0.873485,0.785422,0.886261,0.824767,0.908707,0.784099,0.867151,0.869197,0.878646,...,0.853476,0.790738,0.860699,0.857347,0.846594,0.875682,0.761173,0.773667,0.848829,0.697492
"wap__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.2",0.895624,0.870850,0.794968,0.889368,0.831062,0.914193,0.825166,0.883239,0.896884,0.881614,...,0.885838,0.818881,0.868844,0.858611,0.857438,0.882450,0.783518,0.795008,0.862724,0.779300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
price__number_crossing_m__m_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__number_crossing_m__m_-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
price__count_above__t_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
top_n = 300
corr_cutoff=0.7
feat_list = []
for stock_id in low_corr_list:
    df_top_n = df_features[f'stock={stock_id}'].sort_values(ascending=False).iloc[:top_n]
    print(np.round(df_top_n[0],4), np.round(df_top_n[-1],4))
    feat_list.append(df_top_n.index.tolist())

0.9026 0.5872
0.89 0.5449
0.97 0.4618
0.9133 0.5383
0.8623 0.5546
0.9158 0.6709
1.0 0.4516
0.8945 0.5594
0.9218 0.5897
0.8868 0.642
0.9337 0.7154
0.8765 0.6052
0.8477 0.5819
0.9101 0.5445
0.9633 0.5212
0.8863 0.5171
0.8904 0.5287
0.8881 0.4425
0.8849 0.6541
0.8221 0.5286
0.8297 0.5502
0.8801 0.5928
0.8339 0.3646


In [51]:
over_lap_feat = feat_list[0]
for i in range(1, len(feat_list)):
    over_lap_feat = set(over_lap_feat) & set(feat_list[i])


In [52]:
print(len(over_lap_feat))
print(over_lap_feat)

99
{'price__fft_aggregated__aggtype_"centroid"', 'wap__fft_coefficient__attr_"abs"__coeff_56', 'wap__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0', 'wap__fft_coefficient__attr_"abs"__coeff_25', 'wap__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2', 'wap__fft_coefficient__attr_"abs"__coeff_38', 'wap__fft_coefficient__attr_"abs"__coeff_46', 'price__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4', 'wap__fft_coefficient__attr_"abs"__coeff_19', 'wap__fft_coefficient__attr_"abs"__coeff_18', 'wap__fft_coefficient__attr_"abs"__coeff_37', 'price__fft_coefficient__attr_"abs"__coeff_5', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0', 'wap__fft_aggregated__aggtype_"variance"', 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'wap__fft_coefficient__attr_"abs"__coeff_20', 'price__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'real_vol', 'wap__f

In [53]:
z1_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_1')].index.tolist()
z2_feats = df_features[(df_features.index.isin(over_lap_feat))&(df_features['folder']=='z_2')].index.tolist()

In [54]:
z1_feats = ['stock_id', 'time_id'] + z1_feats
z2_feats = ['stock_id', 'time_id'] + z2_feats
print(f'{z1_feats}\n{z2_feats}')

['stock_id', 'time_id', 'real_vol', 'wap_std', 'wap__mean_abs_change', 'wap__standard_deviation', 'wap__variation_coefficient', 'wap__absolute_sum_of_changes', 'wap__cid_ce__normalize_False', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.2__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.0', 'wap__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'wap__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.4__ql_0.2', 'wap__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2', 'wap__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'wap__change_quantiles__f_agg_"var"__isabs_True__qh_0.8__ql_0.2', 'wap_

In [55]:
z1_list = []
z2_list = []
for stock_id in [2, 20, 40, 69]:
    df = pd.read_csv(f'../data/z_1/{stock_id}', sep='|', compression='bz2', index_col=0)[z1_feats].copy(deep=True)
    print(df.shape)
    z1_list.append(df)
    df = pd.read_csv(f'../data/z_2/{stock_id}', sep='|', compression='bz2', index_col=0)[z2_feats].copy(deep=True)
    print(df.shape)
    z2_list.append(df)

(3830, 60)
(3830, 44)
(3830, 60)
(3830, 44)
(3830, 60)
(3830, 44)
(3830, 60)
(3830, 44)


In [63]:
df_z1=pd.concat(z1_list, axis=0)
df_z2=pd.concat(z2_list, axis=0)
df_z1.shape, df_z2.shape

((15320, 60), (15320, 44))

In [64]:
df_z1z2 = pd.concat([df_z1, df_z2], axis=1)

In [68]:
final_feats = [  'wap__absolute_sum_of_changes',
                 'wap__fft_aggregated__aggtype_"variance"',
                 'wap__spkt_welch_density__coeff_5',
                 'wap__spkt_welch_density__coeff_8',
                 'wap__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',
                 'price__variation_coefficient',
                 'price__mean_abs_change',
                 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
                 'price__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',
                 'price__absolute_sum_of_changes',
              ]

In [69]:
corr_z1z2=df_z1z2.corr()
remaining_feats =  corr_z1z2.index.tolist()
for feat in final_feats:
    if feat in remaining_feats:
        remaining_feats = set(remaining_feats) & set(corr_z1z2[corr_z1z2[feat]<0.8].index)


In [70]:
df_features[df_features.index.isin(remaining_feats)].sort_values(by=['stock=0'],ascending=False)

,stock=0,stock_log=0,stock=1,stock_log=1,stock=2,stock_log=2,stock=3,stock_log=3,stock=4,stock_log=4,...,stock_log=122,stock=123,stock_log=123,stock=124,stock_log=124,stock=125,stock_log=125,stock=126,stock_log=126,folder
feature,,,,,,,,,,,,,,,,,,,,,
"wap__fft_coefficient__attr_""abs""__coeff_37",0.705428,0.635300,0.614951,0.568804,0.728001,0.654779,0.636643,0.575820,0.659793,0.591732,...,0.615942,0.673696,0.613181,0.628550,0.606882,0.717204,0.640338,0.649892,0.603609,z_1
"wap__fft_coefficient__attr_""abs""__coeff_30",0.690356,0.611228,0.619615,0.570912,0.705491,0.657771,0.636517,0.568243,0.663718,0.597505,...,0.612623,0.692488,0.635138,0.626986,0.605637,0.722083,0.639882,0.634087,0.591855,z_1
"wap__fft_coefficient__attr_""abs""__coeff_20",0.684175,0.614173,0.612435,0.568118,0.706912,0.652340,0.622142,0.555402,0.637024,0.584547,...,0.615774,0.698425,0.622527,0.625343,0.602776,0.683528,0.630282,0.621840,0.579542,z_1
"wap__fft_coefficient__attr_""abs""__coeff_65",0.683252,0.615163,0.639198,0.588337,0.679561,0.649047,0.652481,0.583322,0.640191,0.581496,...,0.640707,0.690581,0.628722,0.624964,0.603745,0.690985,0.633529,0.646391,0.607742,z_1
"wap__fft_coefficient__attr_""abs""__coeff_18",0.681977,0.609285,0.597072,0.544624,0.715615,0.666001,0.641402,0.564815,0.630166,0.577930,...,0.619601,0.649056,0.599935,0.629651,0.603618,0.662051,0.619935,0.631080,0.584960,z_1
"wap__fft_coefficient__attr_""abs""__coeff_56",0.679962,0.618701,0.644914,0.590148,0.738463,0.673600,0.655672,0.572366,0.626958,0.574983,...,0.623004,0.680678,0.627776,0.619404,0.600030,0.683263,0.647581,0.653746,0.612193,z_1
"wap__fft_coefficient__attr_""abs""__coeff_24",0.679318,0.608761,0.607979,0.560948,0.664703,0.643559,0.630969,0.573207,0.617912,0.571710,...,0.610578,0.660230,0.604309,0.626722,0.608777,0.680273,0.621885,0.631669,0.584054,z_1
"wap__fft_coefficient__attr_""abs""__coeff_42",0.679011,0.631075,0.619288,0.570421,0.715760,0.654392,0.647281,0.570417,0.631950,0.579615,...,0.615606,0.676639,0.608634,0.631995,0.604917,0.660708,0.634154,0.652427,0.604114,z_1
"wap__fft_coefficient__attr_""abs""__coeff_34",0.677689,0.605664,0.612554,0.558134,0.673822,0.650399,0.614726,0.566176,0.644392,0.586033,...,0.614984,0.665519,0.597598,0.618719,0.602133,0.680423,0.625770,0.645371,0.590974,z_1


### combine data for training

In [71]:
final_feats = [  'wap__absolute_sum_of_changes',
                 'wap__fft_aggregated__aggtype_"variance"',
                 'wap__spkt_welch_density__coeff_5',
                 'wap__spkt_welch_density__coeff_8',
                 'wap__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2',
                 'price__variation_coefficient',
                 'price__mean_abs_change',
                 'price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6',
                 'price__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',
                 'price__absolute_sum_of_changes',
              ]

In [72]:
z1_feats = set(df_features[df_features['folder']=='z_1'].index.tolist()) & set(final_feats)
z2_feats = set(df_features[df_features['folder']=='z_2'].index.tolist()) & set(final_feats)

In [73]:
print(len(z1_feats|z2_feats)-len(final_feats))

0


In [74]:
z1_feats = ['stock_id', 'time_id'] + list(z1_feats)
z2_feats = ['stock_id', 'time_id'] + list(z2_feats)

In [75]:
len(final_feats)

10

In [76]:
df_train.head(3)

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168


In [77]:
df_train.shape

(428932, 3)

In [78]:
df_list = []

for stock_id  in stock_list:
    df_z1 = pd.read_csv(f'../data/z_1/{stock_id}', sep='|', compression='bz2', index_col=0)[z1_feats].copy(deep=True)
    df_z2 = pd.read_csv(f'../data/z_2/{stock_id}', sep='|', compression='bz2', index_col=0)[z2_feats].copy(deep=True)
    df = df_z1.merge(df_z2, on=['stock_id', 'time_id'], how='left')
    print(stock_id, df_z1.shape, df_z2.shape, df.shape)
    df_list.append(df)

0 (3830, 7) (3830, 7) (3830, 12)
1 (3830, 7) (3830, 7) (3830, 12)
2 (3830, 7) (3830, 7) (3830, 12)
3 (3830, 7) (3830, 7) (3830, 12)
4 (3830, 7) (3830, 7) (3830, 12)
5 (3830, 7) (3830, 7) (3830, 12)
6 (3830, 7) (3830, 7) (3830, 12)
7 (3830, 7) (3830, 7) (3830, 12)
8 (3830, 7) (3830, 7) (3830, 12)
9 (3830, 7) (3830, 7) (3830, 12)
10 (3830, 7) (3830, 7) (3830, 12)
11 (3830, 7) (3830, 7) (3830, 12)
13 (3829, 7) (3829, 7) (3829, 12)
14 (3830, 7) (3830, 7) (3830, 12)
15 (3830, 7) (3830, 7) (3830, 12)
16 (3830, 7) (3830, 7) (3830, 12)
17 (3830, 7) (3830, 7) (3830, 12)
18 (3830, 7) (3827, 7) (3830, 12)
19 (3830, 7) (3830, 7) (3830, 12)
20 (3830, 7) (3830, 7) (3830, 12)
21 (3830, 7) (3830, 7) (3830, 12)
22 (3830, 7) (3830, 7) (3830, 12)
23 (3830, 7) (3830, 7) (3830, 12)
26 (3830, 7) (3830, 7) (3830, 12)
27 (3830, 7) (3827, 7) (3830, 12)
28 (3830, 7) (3830, 7) (3830, 12)
29 (3830, 7) (3830, 7) (3830, 12)
30 (3830, 7) (3830, 7) (3830, 12)
31 (3830, 7) (3774, 7) (3830, 12)
32 (3830, 7) (3830, 7) (

In [79]:
len(stock_list)

112

In [80]:
df_all = pd.concat(df_list, axis=0)

In [81]:
df_all.shape, df_train.shape

((428932, 12), (428932, 3))

In [82]:
df_all = df_train.merge(df_all, on=['stock_id', 'time_id'], how='inner')

In [83]:
df_all.shape

(428932, 13)

In [85]:
df_all.to_csv(f'../data/y_1/{len(final_feats)}features', compression='bz2', sep='|', index=False)

In [89]:
df_all[final_feats].isna().sum()

wap__absolute_sum_of_changes                                          0
wap__fft_aggregated__aggtype_"variance"                               0
wap__spkt_welch_density__coeff_5                                      0
wap__spkt_welch_density__coeff_8                                      0
wap__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.2        0
price__variation_coefficient                                         80
price__mean_abs_change                                               80
price__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.6    80
price__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6     80
price__absolute_sum_of_changes                                       80
dtype: int64

In [87]:
print(f'{high_corr_list}\n{low_corr_list}')

[10, 11, 13, 14, 15, 23, 29, 32, 35, 42, 43, 50, 52, 62, 69, 70, 72, 73, 76, 87, 93, 95, 101, 108, 109, 119, 122, 126]
[2, 13, 19, 20, 21, 29, 30, 34, 39, 41, 43, 46, 47, 53, 58, 64, 67, 68, 69, 80, 81, 99, 110]


In [91]:
df_all[df_all['stock_id'].isin(high_corr_list)].to_csv(f'../data/y_1/{len(final_feats)}features_highcorr', compression='bz2', sep='|', index=False)


In [92]:
df_all[df_all['stock_id'].isin(low_corr_list)].to_csv(f'../data/y_1/{len(final_feats)}features_lowcorr', compression='bz2', sep='|', index=False)
